#
<FONT size="10pt"> <center>API N°3 : Les APIs de Montpellier</center></FONT>. 

In [1]:
import requests
import json
import pandas as pd
import numpy as np
import tqdm
import random
import folium
from folium.plugins import DualMap

from IPython.display import display
pd.options.display.max_columns = None

# 1 : API place de parking 

In [2]:
# Lien url de la request : 
link = 'https://portail-api-data.montpellier3m.fr/offstreetparking?limit=1000'
    
# Request du lien : 
r = requests.get(link)

# Vérificiation de la bonne lecture de l'API : (200 = Good) :
r

<Response [200]>

In [3]:
# Lecture de la request en json :
Parkings = r.json()

# Visualiser un fichier json de façon plus structuré :
Parkings = pd.json_normalize(Parkings)

# Data Frame :
Parkings.head()

,id,type,allowedVehicleType.type,allowedVehicleType.value,availableSpotNumber.type,availableSpotNumber.value,availableSpotNumber.metadata.timestamp.type,availableSpotNumber.metadata.timestamp.value,category.type,category.value,layout.type,layout.value,location.type,location.value.type,location.value.coordinates,name.type,name.value,requiredPermit.type,requiredPermit.value,status.type,status.value,status.metadata.timestamp.type,status.metadata.timestamp.value,totalSpotNumber.type,totalSpotNumber.value
0,urn:ngsi-ld:parking:001,OffStreetParking,StructuredValue,"[car, moped]",Number,116,DateTime,2024-02-21T14:38:50.000Z,StructuredValue,"[feeCharged, barrierAccess, underground]",StructuredValue,[multiLevel],geo:json,Point,"[3.88881893, 43.60871606]",Text,Antigone,Text,noPermitNeeded,Text,Open,DateTime,2024-02-21T14:38:50.000Z,Number,239
1,urn:ngsi-ld:parking:003,OffStreetParking,StructuredValue,"[car, moped]",Number,300,DateTime,2024-02-21T14:35:31.000Z,StructuredValue,"[feeCharged, barrierAccess, underground]",StructuredValue,[multiLevel],geo:json,Point,"[3.87976196, 43.60856092]",Text,Comedie,Text,noPermitNeeded,Text,Open,DateTime,2024-02-21T14:35:31.000Z,Number,739
2,urn:ngsi-ld:parking:004,OffStreetParking,StructuredValue,"[car, moped]",Number,227,DateTime,2024-02-21T14:38:54.000Z,StructuredValue,"[feeCharged, barrierAccess, underground]",StructuredValue,[multiLevel],geo:json,Point,"[3.88225773, 43.61388821]",Text,Corum,Text,noPermitNeeded,Text,Open,DateTime,2024-02-21T14:38:54.000Z,Number,330
3,urn:ngsi-ld:parking:005,OffStreetParking,StructuredValue,"[car, moped]",Number,321,DateTime,2024-02-21T14:38:59.000Z,StructuredValue,"[feeCharged, barrierAccess, underground]",StructuredValue,[multiLevel],geo:json,Point,"[3.89253074, 43.60784971]",Text,Europa,Text,noPermitNeeded,Text,Open,DateTime,2024-02-21T14:38:59.000Z,Number,591
4,urn:ngsi-ld:parking:006,OffStreetParking,StructuredValue,"[car, moped]",Number,223,DateTime,2024-02-21T14:34:57.000Z,StructuredValue,"[feeCharged, barrierAccess, underground]",StructuredValue,[multiLevel],geo:json,Point,"[3.87657084, 43.61074912]",Text,Foch,Text,noPermitNeeded,Text,Open,DateTime,2024-02-21T14:34:57.000Z,Number,646


In [4]:
# Nettoyage de la DF :

# Garder les colonnes utiles avec une cardinalité sup à 1 :
Parkings = Parkings[['id', 'name.value', 'location.value.coordinates', 'availableSpotNumber.value', 'totalSpotNumber.value']]

# Extraire l'id des parking et la réordonner : 
Parkings['id'] = Parkings['id'].apply(lambda x : x.split(':')[-1])
Parkings = Parkings.sort_values('id').reset_index(drop = True)

# Renommer les colonnes :
Parkings.columns = ['Ids', 'Noms', 'Localisations', 'PlacesDisponibles', 'PlacesTotales']

# Création d'une colonne % de dispo :
Parkings["PourcentDisponibilite"] = Parkings["PlacesDisponibles"] * 100 / Parkings["PlacesTotales"]
Parkings["PourcentDisponibilite"] = Parkings["PourcentDisponibilite"].astype('int64')

# Inversion des coordonnées pour faire une carte :
Parkings['Localisations'] = Parkings['Localisations'].apply(lambda x : x[::-1])

# DF finale :
Parkings

,Ids,Noms,Localisations,PlacesDisponibles,PlacesTotales,PourcentDisponibilite
0,001,Antigone,"[43.60871606, 3.88881893]",116,239,48
1,002,Arc de Triomphe,"[43.61100267, 3.8732007]",66,440,15
2,003,Comedie,"[43.60856092, 3.87976196]",300,739,40
3,004,Corum,"[43.61388821, 3.88225773]",227,330,68
4,005,Europa,"[43.60784971, 3.89253074]",321,591,54
5,006,Foch,"[43.61074912, 3.87657084]",223,646,34
6,007,Gambetta,"[43.60695138, 3.87137436]",191,452,42
7,008,Gare,"[43.604024, 3.87891]",262,649,40
8,009,Triangle,"[43.60923384, 3.88184418]",305,434,70
9,010,Pitot,"[43.61224494, 3.87019117]",349,587,59


# 2 : API place de velos

In [5]:
# Lien url de la request : 
link = 'https://portail-api-data.montpellier3m.fr/bikestation?limit=1000'
    
# Request du lien : 
r = requests.get(link)

# Vérificiation de la bonne lecture de l'API : (200 = Good) :
r

<Response [200]>

In [6]:
# Lecture de la request en json :
Velos = r.json()

# Visualiser un fichier json de façon plus structuré :
Velos = pd.json_normalize(Velos)

# Data Frame :
Velos.head()

,id,type,address.type,address.value.addressCountry,address.value.addressLocality,address.value.streetAddress,availableBikeNumber.type,availableBikeNumber.value,availableBikeNumber.metadata.timestamp.type,availableBikeNumber.metadata.timestamp.value,freeSlotNumber.type,freeSlotNumber.value,location.type,location.value.type,location.value.coordinates,status.type,status.value,totalSlotNumber.type,totalSlotNumber.value
0,urn:ngsi-ld:station:001,BikeHireDockingStation,PostalAddress,FR,Montpellier,Rue Jules Ferry - Gare Saint-Roch,Number,9,DateTime,2024-02-21T14:42:17.000Z,Number,3,geo:json,Point,"[3.881346, 43.605366]",Text,working,Number,12
1,urn:ngsi-ld:station:002,BikeHireDockingStation,PostalAddress,FR,Montpellier,Comédie,Number,11,DateTime,2024-02-21T14:42:17.000Z,Number,9,geo:json,Point,"[3.878778, 43.608148]",Text,working,Number,20
2,urn:ngsi-ld:station:004,BikeHireDockingStation,PostalAddress,FR,Montpellier,Hôtel de Ville,Number,2,DateTime,2024-02-21T14:42:17.000Z,Number,14,geo:json,Point,"[3.894866, 43.599088]",Text,working,Number,16
3,urn:ngsi-ld:station:005,BikeHireDockingStation,PostalAddress,FR,Montpellier,Corum,Number,4,DateTime,2024-02-21T14:42:17.000Z,Number,8,geo:json,Point,"[3.8816, 43.613989]",Text,working,Number,12
4,urn:ngsi-ld:station:006,BikeHireDockingStation,PostalAddress,FR,Montpellier,Place Albert 1er - St Charles,Number,9,DateTime,2024-02-21T14:42:17.000Z,Number,18,geo:json,Point,"[3.873375, 43.616768]",Text,working,Number,27


In [7]:
# Nettoyage de la DF :

# Garder les colonnes utiles avec une cardinalité sup à 1 :
Velos = Velos[['id', 'address.value.streetAddress', 'availableBikeNumber.value', 'freeSlotNumber.value',
       'location.value.coordinates', 'totalSlotNumber.value']]

# Extraire l'id des Velos et la réordonner : 
Velos['id'] = Velos['id'].apply(lambda x : x.split(':')[-1])
Velos = Velos.sort_values('id').reset_index(drop = True)

# Renommer les colonnes :
Velos.columns = ['Ids', 'Adresses', 'NbVelosDisponibles', 'NbPlacesVides', 'Localisations', 'NbPlacesTotales']

# Création d'une colonne %Velos de dispo :
Velos["PourcentVelosDisponibilite"] = Velos["NbVelosDisponibles"] * 100 / Velos["NbPlacesTotales"]
Velos["PourcentVelosDisponibilite"] = Velos["PourcentVelosDisponibilite"].astype('int64')

# Création d'une colonne %Velos de dispo :
Velos["PourcentVelosDisponibilite"] = Velos["NbVelosDisponibles"] * 100 / Velos["NbPlacesTotales"]
Velos["PourcentVelosDisponibilite"] = Velos["PourcentVelosDisponibilite"].astype('int64')

# Création d'une colonne %Places Velos de dispo :
Velos["PourcentPlacesDisponibilite"] = Velos["NbPlacesVides"] * 100 / Velos["NbPlacesTotales"]
Velos["PourcentPlacesDisponibilite"] = Velos["PourcentPlacesDisponibilite"].astype('int64')

# Inversion des coordonnées pour faire une carte :
Velos['Localisations'] = Velos['Localisations'].apply(lambda x : x[::-1])

# DF finale :
Velos

,Ids,Adresses,NbVelosDisponibles,NbPlacesVides,Localisations,NbPlacesTotales,PourcentVelosDisponibilite,PourcentPlacesDisponibilite
0,001,Rue Jules Ferry - Gare Saint-Roch,9,3,"[43.605366, 3.881346]",12,75,25
1,002,Comédie,11,9,"[43.608148, 3.878778]",20,55,45
2,004,Hôtel de Ville,2,14,"[43.599088, 3.894866]",16,12,87
3,005,Corum,4,8,"[43.613989, 3.8816]",12,33,66
4,006,Place Albert 1er - St Charles,9,18,"[43.616768, 3.873375]",27,33,66
5,007,Foch,6,2,"[43.610989, 3.873345]",8,75,25
6,008,Halles Castellane,5,7,"[43.609935, 3.877208]",12,41,58
7,009,Observatoire,5,3,"[43.606301, 3.87724]",8,62,37
8,010,Rondelet,14,2,"[43.603038, 3.875796]",16,87,12
9,011,Plan Cabanes,8,4,"[43.608491, 3.868389]",12,66,33


# 3 : Cartes Folium

In [8]:
# Couleurs des icons :
def color(remplissage):
    if remplissage <= 5:
        col = 'darkred'
    elif remplissage <= 20:
        col = 'orange'
    else:
        col='green'
    return col

# Petite carte folium :
m = folium.Map(location = Parkings.Localisations[0], zoom_start=13)

for i in range(len(Parkings)): 
    folium.Marker(
        location = Parkings.Localisations[i], 
        icon=folium.Icon(color=color(Parkings['PourcentDisponibilite'][i]), icon='square-parking', prefix='fa'),
    ).add_to(m)

m

In [9]:
Velos

,Ids,Adresses,NbVelosDisponibles,NbPlacesVides,Localisations,NbPlacesTotales,PourcentVelosDisponibilite,PourcentPlacesDisponibilite
0,001,Rue Jules Ferry - Gare Saint-Roch,9,3,"[43.605366, 3.881346]",12,75,25
1,002,Comédie,11,9,"[43.608148, 3.878778]",20,55,45
2,004,Hôtel de Ville,2,14,"[43.599088, 3.894866]",16,12,87
3,005,Corum,4,8,"[43.613989, 3.8816]",12,33,66
4,006,Place Albert 1er - St Charles,9,18,"[43.616768, 3.873375]",27,33,66
5,007,Foch,6,2,"[43.610989, 3.873345]",8,75,25
6,008,Halles Castellane,5,7,"[43.609935, 3.877208]",12,41,58
7,009,Observatoire,5,3,"[43.606301, 3.87724]",8,62,37
8,010,Rondelet,14,2,"[43.603038, 3.875796]",16,87,12
9,011,Plan Cabanes,8,4,"[43.608491, 3.868389]",12,66,33


In [10]:
# Double map pour voir les vélos disponibles vs les places disponibles :
m = folium.plugins.DualMap(Parkings.Localisations[0], tiles="openstreetmap", zoom_start=15)

for i in range(len(Velos)): 
    folium.Marker(
        location = Velos.Localisations[i], 
        icon=folium.Icon(color=color(Velos['PourcentVelosDisponibilite'][i]), icon = str(Velos['NbVelosDisponibles'][i]), prefix='fa'),
    ).add_to(m.m1)
    
for i in range(len(Velos)): 
    folium.Marker(
        location = Velos.Localisations[i], 
        icon=folium.Icon(color=color(Velos['PourcentPlacesDisponibilite'][i]), icon = str(Velos['NbPlacesVides'][i]), prefix='fa'),
    ).add_to(m.m2)

folium.LayerControl(collapsed=False).add_to(m)
m